# Chapter 36: Mocking with unittest.mock

This notebook covers Python's `unittest.mock` module, which provides tools to replace parts of your system with mock objects during testing. Mocking is essential for isolating the code under test from its dependencies.

## Key Concepts
- **Mock**: A flexible object that records calls and can be configured to return specific values
- **MagicMock**: A Mock subclass with default implementations of dunder methods
- **patch**: A decorator/context manager that temporarily replaces objects with mocks
- **side_effect**: Configure a mock to raise exceptions or return values from an iterable
- **spec**: Restrict a mock to only allow attributes that exist on a real object
- **Call tracking**: Inspect how mocks were called (arguments, count, order)

## Section 1: Creating Basic Mocks

A `Mock` object records every interaction. You can call it, access attributes, and later inspect exactly what happened.

In [ ]:
from unittest.mock import Mock

# Create a basic mock
m: Mock = Mock()

# Calling a mock returns another Mock by default
result = m()
print(f"Calling Mock(): {result}")
print(f"Type of result: {type(result)}")

# Accessing attributes creates child mocks automatically
print(f"\nm.some_attr: {m.some_attr}")
print(f"m.nested.deep.attr: {m.nested.deep.attr}")

# Each attribute access returns a consistent mock
print(f"\nm.some_attr is m.some_attr: {m.some_attr is m.some_attr}")

In [ ]:
from unittest.mock import Mock

# Mock with a name (helpful for debugging)
db_mock: Mock = Mock(name="database")
print(f"Named mock: {db_mock}")

# Calling with arguments -- the mock records everything
db_mock.query("SELECT * FROM users", limit=10)
db_mock.insert({"name": "Alice", "age": 30})

# Inspect what was called
print(f"\ndb_mock.query called: {db_mock.query.called}")
print(f"db_mock.insert called: {db_mock.insert.called}")
print(f"db_mock.delete called: {db_mock.delete.called}")

## Section 2: Configuring Return Values

Use `return_value` to control what a mock returns when called. This lets you simulate specific responses from dependencies.

In [ ]:
from unittest.mock import Mock

# Set return_value at creation
m: Mock = Mock(return_value=42)
result: int = m()
print(f"Mock with return_value=42: m() = {result}")
assert result == 42

# Set return_value after creation
m.return_value = "hello"
print(f"After changing return_value: m() = {m()}")

# Return value on a method
api: Mock = Mock()
api.get_user.return_value = {"id": 1, "name": "Alice"}
api.get_users.return_value = [{"id": 1}, {"id": 2}]

user: dict[str, int | str] = api.get_user(1)
users: list[dict[str, int]] = api.get_users()

print(f"\napi.get_user(1) = {user}")
print(f"api.get_users() = {users}")

In [ ]:
from unittest.mock import Mock


# Practical example: testing code that depends on an external service
class PaymentProcessor:
    """Processes payments via an external gateway."""

    def __init__(self, gateway: object) -> None:
        self.gateway = gateway

    def charge(self, amount: float, card_token: str) -> dict[str, object]:
        """Charge a card and return the result."""
        response = self.gateway.create_charge(  # type: ignore[attr-defined]
            amount=amount, token=card_token
        )
        return {"success": response["status"] == "ok", "id": response["charge_id"]}


# Create a mock gateway
mock_gateway: Mock = Mock()
mock_gateway.create_charge.return_value = {"status": "ok", "charge_id": "ch_123"}

# Test the processor with the mock
processor: PaymentProcessor = PaymentProcessor(mock_gateway)
result: dict[str, object] = processor.charge(29.99, "tok_abc")

print(f"Charge result: {result}")
assert result == {"success": True, "id": "ch_123"}
print("PaymentProcessor test PASSED")

## Section 3: side_effect -- Exceptions and Iterables

`side_effect` provides more dynamic behavior than `return_value`. It can raise exceptions, return values from an iterable, or run a custom function.

In [ ]:
from unittest.mock import Mock

# side_effect with an exception
m: Mock = Mock(side_effect=ValueError("boom"))

try:
    m()
except ValueError as e:
    print(f"Caught exception: {e}")

# side_effect with an iterable -- returns values in sequence
m = Mock(side_effect=[1, 2, 3])
print(f"\nFirst call:  {m()}")
print(f"Second call: {m()}")
print(f"Third call:  {m()}")

# StopIteration when iterable is exhausted
try:
    m()
except StopIteration:
    print("Fourth call: StopIteration (iterable exhausted)")

In [ ]:
from unittest.mock import Mock


# side_effect with a function -- gets called with the same arguments
def double(x: int) -> int:
    return x * 2


m: Mock = Mock(side_effect=double)
print(f"m(5) = {m(5)}")
print(f"m(10) = {m(10)}")
print(f"m(0) = {m(0)}")

# Mixing exceptions in an iterable
m = Mock(side_effect=[10, ValueError("retry failed"), 30])

print(f"\nCall 1: {m()}")
try:
    m()
except ValueError as e:
    print(f"Call 2: raised ValueError({e})")
print(f"Call 3: {m()}")

In [ ]:
from unittest.mock import Mock


# Practical example: simulating network retries
class ApiClient:
    """HTTP client that retries on failure."""

    def __init__(self, http: object) -> None:
        self.http = http

    def fetch_with_retry(self, url: str, max_retries: int = 3) -> str:
        """Fetch URL with retries on connection error."""
        for attempt in range(max_retries):
            try:
                return self.http.get(url)  # type: ignore[attr-defined]
            except ConnectionError:
                if attempt == max_retries - 1:
                    raise
        return ""  # unreachable, satisfies type checker


# Mock HTTP that fails twice then succeeds
mock_http: Mock = Mock()
mock_http.get.side_effect = [
    ConnectionError("timeout"),
    ConnectionError("timeout"),
    "<html>Success</html>",
]

client: ApiClient = ApiClient(mock_http)
result: str = client.fetch_with_retry("https://example.com")

print(f"Result: {result}")
print(f"Total attempts: {mock_http.get.call_count}")
assert result == "<html>Success</html>"
assert mock_http.get.call_count == 3
print("Retry test PASSED")

## Section 4: Call Tracking and Assertions

Mocks record every call made to them, including arguments. You can assert exactly how the mock was called.

In [ ]:
from unittest.mock import Mock, call

m: Mock = Mock()

# Make several calls
m("first")
m("second", key="value")
m("third")

# call_count tracks total calls
print(f"call_count: {m.call_count}")
assert m.call_count == 3

# called is True if mock was called at least once
print(f"called: {m.called}")

# call_args is the most recent call
print(f"call_args (last call): {m.call_args}")

# call_args_list is the full history
print(f"call_args_list: {m.call_args_list}")

# Assert specific calls
m.assert_called_with("third")  # Checks last call only
print("\nassert_called_with('third'): PASSED")

m.assert_any_call("second", key="value")  # Checks if this call happened at all
print("assert_any_call('second', key='value'): PASSED")

In [ ]:
from unittest.mock import Mock, call

# assert_called_once_with -- must have been called exactly once
m: Mock = Mock()
m(42, name="test")

m.assert_called_once_with(42, name="test")
print("assert_called_once_with(42, name='test'): PASSED")

# assert_has_calls -- verify a sequence of calls
logger: Mock = Mock()
logger.info("Starting")
logger.debug("Processing item 1")
logger.debug("Processing item 2")
logger.info("Done")

expected_calls: list[call] = [
    call.info("Starting"),
    call.debug("Processing item 1"),
    call.debug("Processing item 2"),
    call.info("Done"),
]
logger.assert_has_calls(expected_calls)
print("\nassert_has_calls with exact order: PASSED")

# With any_order=True, calls can appear in any order
logger.assert_has_calls(
    [call.info("Done"), call.info("Starting")],
    any_order=True,
)
print("assert_has_calls with any_order=True: PASSED")

In [ ]:
from unittest.mock import Mock

# assert_not_called -- verify mock was never called
m: Mock = Mock()
m.assert_not_called()
print("assert_not_called on fresh mock: PASSED")

# reset_mock -- clear all call history
m("first")
m("second")
print(f"\nBefore reset: call_count = {m.call_count}")

m.reset_mock()
print(f"After reset: call_count = {m.call_count}")
print(f"After reset: called = {m.called}")
print(f"After reset: call_args_list = {m.call_args_list}")

## Section 5: MagicMock -- Mocking Dunder Methods

`MagicMock` is a subclass of `Mock` that provides default implementations of magic (dunder) methods like `__len__`, `__iter__`, `__getitem__`, and more.

In [ ]:
from unittest.mock import MagicMock, Mock

# Regular Mock does not support dunder methods out of the box
# MagicMock does

m: MagicMock = MagicMock()

# __len__
m.__len__.return_value = 5
print(f"len(m) = {len(m)}")
assert len(m) == 5

# __bool__
m.__bool__.return_value = False
print(f"bool(m) = {bool(m)}")
assert bool(m) is False

# __str__
m.__str__.return_value = "custom string"
print(f"str(m) = {str(m)}")

# __iter__
m.__iter__.return_value = iter([1, 2, 3])
items: list[int] = list(m)
print(f"list(m) = {items}")

In [ ]:
from unittest.mock import MagicMock

# MagicMock as a context manager
m: MagicMock = MagicMock()
m.__enter__.return_value = "resource"

with m as resource:
    print(f"Context manager yielded: {resource}")

m.__enter__.assert_called_once()
m.__exit__.assert_called_once()
print("Context manager protocol verified")

# MagicMock with __getitem__
config: MagicMock = MagicMock()
config.__getitem__.side_effect = lambda key: {"host": "localhost", "port": 8080}[key]

print(f"\nconfig['host'] = {config['host']}")
print(f"config['port'] = {config['port']}")

## Section 6: patch -- Replacing Objects During Tests

`patch` temporarily replaces an object (function, class, attribute) with a mock for the duration of a test. It can be used as a decorator or context manager.

In [ ]:
from unittest.mock import patch
import os

# patch as a context manager
print(f"Before patch: os.getcwd() = {os.getcwd()}")

with patch("os.getcwd", return_value="/fake/path"):
    print(f"During patch: os.getcwd() = {os.getcwd()}")
    assert os.getcwd() == "/fake/path"

print(f"After patch: os.getcwd() = {os.getcwd()}")
print("\npatch restored original behavior automatically")

In [ ]:
from unittest.mock import patch, Mock
import os


# Patching a module-level function
def get_home_directory() -> str:
    """Get the user's home directory."""
    return os.path.expanduser("~")


print(f"Real home: {get_home_directory()}")

with patch("os.path.expanduser", return_value="/mock/home"):
    print(f"Mocked home: {get_home_directory()}")
    assert get_home_directory() == "/mock/home"

print(f"Restored home: {get_home_directory()}")

In [ ]:
from unittest.mock import patch, MagicMock


# patch.object -- patch an attribute on a specific object
class EmailService:
    def send(self, to: str, subject: str, body: str) -> bool:
        """Send a real email (expensive/slow operation)."""
        raise NotImplementedError("Real email sending not available")


class UserRegistration:
    def __init__(self, email_service: EmailService) -> None:
        self.email_service: EmailService = email_service

    def register(self, username: str, email: str) -> dict[str, str]:
        """Register a user and send a welcome email."""
        user: dict[str, str] = {"username": username, "email": email}
        self.email_service.send(
            to=email,
            subject="Welcome!",
            body=f"Hello {username}, welcome aboard!",
        )
        return user


email_svc: EmailService = EmailService()
reg: UserRegistration = UserRegistration(email_svc)

# Patch the send method to avoid real email sending
with patch.object(email_svc, "send", return_value=True) as mock_send:
    user: dict[str, str] = reg.register("alice", "alice@example.com")
    print(f"Registered user: {user}")

    # Verify the email service was called correctly
    mock_send.assert_called_once_with(
        to="alice@example.com",
        subject="Welcome!",
        body="Hello alice, welcome aboard!",
    )
    print("Email service called with correct arguments")
    print("Test PASSED")

In [ ]:
from unittest.mock import patch

# patch.dict -- temporarily modify a dictionary
import os

original_path: str | None = os.environ.get("MY_APP_SECRET")
print(f"Before: MY_APP_SECRET = {original_path!r}")

with patch.dict(os.environ, {"MY_APP_SECRET": "s3cret", "DEBUG": "true"}):
    print(f"During: MY_APP_SECRET = {os.environ['MY_APP_SECRET']!r}")
    print(f"During: DEBUG = {os.environ['DEBUG']!r}")

print(f"After: MY_APP_SECRET = {os.environ.get('MY_APP_SECRET')!r}")
print(f"After: DEBUG = {os.environ.get('DEBUG')!r}")
print("\npatch.dict restored original environment")

## Section 7: spec -- Constraining Mocks

Without `spec`, a mock will accept any attribute access or method call, which can hide bugs. The `spec` parameter restricts the mock to only allow attributes that exist on the real object.

In [ ]:
from unittest.mock import Mock

# Without spec -- any attribute works (can hide typos)
m: Mock = Mock()
m.append(1)       # Fine, even though 'm' isn't a list
m.nonexistent()   # Fine, no error
print(f"Without spec: m.nonexistent() = {m.nonexistent()}")

# With spec -- restricted to real object's interface
m_list: Mock = Mock(spec=list)
m_list.append(1)  # Valid: list has append
print(f"\nWith spec=list: m_list.append(1) works")

try:
    m_list.nonexistent()  # type: ignore[attr-defined]
except AttributeError as e:
    print(f"With spec=list: m_list.nonexistent() -> AttributeError: {e}")

In [ ]:
from unittest.mock import Mock, create_autospec


class Calculator:
    """A simple calculator."""

    def add(self, a: float, b: float) -> float:
        return a + b

    def divide(self, a: float, b: float) -> float:
        if b == 0:
            raise ZeroDivisionError("Cannot divide by zero")
        return a / b


# spec restricts attribute access
mock_calc: Mock = Mock(spec=Calculator)
mock_calc.add.return_value = 10.0
print(f"mock_calc.add(3, 7) = {mock_calc.add(3, 7)}")

# create_autospec also checks argument signatures
auto_calc = create_autospec(Calculator)
auto_calc.add.return_value = 10.0
print(f"auto_calc.add(3, 7) = {auto_calc.add(3, 7)}")

# Wrong number of arguments raises TypeError
try:
    auto_calc.add(1, 2, 3)  # Too many args
except TypeError as e:
    print(f"\nauto_calc.add(1, 2, 3) -> TypeError: {e}")

# Non-existent method raises AttributeError
try:
    auto_calc.multiply(2, 3)  # type: ignore[attr-defined]
except AttributeError as e:
    print(f"auto_calc.multiply(2, 3) -> AttributeError: {e}")

## Section 8: Putting It All Together

A complete example combining multiple mocking techniques to test a realistic component.

In [ ]:
from unittest.mock import Mock, patch, call, MagicMock


# System under test: an order processing pipeline
class OrderProcessor:
    """Processes customer orders using external services."""

    def __init__(
        self,
        inventory: object,
        payment: object,
        notifier: object,
    ) -> None:
        self.inventory = inventory
        self.payment = payment
        self.notifier = notifier

    def process_order(
        self, item_id: str, quantity: int, card_token: str, email: str
    ) -> dict[str, object]:
        """Process an order: check stock, charge card, send confirmation."""
        # Check inventory
        stock: int = self.inventory.check_stock(item_id)  # type: ignore[attr-defined]
        if stock < quantity:
            raise ValueError(f"Insufficient stock: {stock} < {quantity}")

        # Charge payment
        charge = self.payment.charge(  # type: ignore[attr-defined]
            amount=quantity * 9.99, token=card_token
        )

        # Reserve inventory
        self.inventory.reserve(item_id, quantity)  # type: ignore[attr-defined]

        # Send confirmation
        self.notifier.send_email(  # type: ignore[attr-defined]
            to=email, subject="Order Confirmed", body=f"Order {charge['id']} confirmed"
        )

        return {"order_id": charge["id"], "quantity": quantity, "status": "confirmed"}


# Create mocks for all dependencies
mock_inventory: Mock = Mock()
mock_payment: Mock = Mock()
mock_notifier: Mock = Mock()

# Configure mock responses
mock_inventory.check_stock.return_value = 50
mock_payment.charge.return_value = {"id": "ord_789", "status": "ok"}

# Create processor and run test
processor: OrderProcessor = OrderProcessor(mock_inventory, mock_payment, mock_notifier)
result: dict[str, object] = processor.process_order(
    item_id="SKU_001", quantity=3, card_token="tok_xyz", email="bob@test.com"
)

# Verify result
print(f"Order result: {result}")
assert result == {"order_id": "ord_789", "quantity": 3, "status": "confirmed"}

# Verify interactions
mock_inventory.check_stock.assert_called_once_with("SKU_001")
mock_inventory.reserve.assert_called_once_with("SKU_001", 3)
mock_payment.charge.assert_called_once_with(amount=29.97, token="tok_xyz")
mock_notifier.send_email.assert_called_once_with(
    to="bob@test.com",
    subject="Order Confirmed",
    body="Order ord_789 confirmed",
)

print("\nAll mock interactions verified:")
print(f"  inventory.check_stock calls: {mock_inventory.check_stock.call_count}")
print(f"  inventory.reserve calls: {mock_inventory.reserve.call_count}")
print(f"  payment.charge calls: {mock_payment.charge.call_count}")
print(f"  notifier.send_email calls: {mock_notifier.send_email.call_count}")
print("\nOrder processing test PASSED")

## Summary

### Mock and MagicMock
- **`Mock()`**: Creates a callable mock that records all interactions
- **`MagicMock()`**: A Mock subclass with default implementations of dunder methods (`__len__`, `__iter__`, `__getitem__`, etc.)
- **`Mock(name="...")`**: Give a mock a name for better debugging output

### Configuring Behavior
- **`return_value`**: Set what the mock returns when called (`Mock(return_value=42)`)
- **`side_effect`**: Raise exceptions (`side_effect=ValueError(...)`), return from iterable (`side_effect=[1, 2, 3]`), or call a function (`side_effect=my_func`)

### Call Tracking
- **`m.called`**: Boolean -- was the mock called?
- **`m.call_count`**: Integer -- how many times was it called?
- **`m.call_args`**: The arguments of the most recent call
- **`m.call_args_list`**: Complete history of all calls

### Assertion Methods
- **`assert_called_once_with(...)`**: Called exactly once with these arguments
- **`assert_called_with(...)`**: Last call had these arguments
- **`assert_any_call(...)`**: This call appeared at least once
- **`assert_has_calls([...], any_order=False)`**: Verify a sequence of calls
- **`assert_not_called()`**: Mock was never called

### patch
- **`patch("module.attr")`**: Replace an attribute with a mock (context manager or decorator)
- **`patch.object(obj, "attr")`**: Replace an attribute on a specific object
- **`patch.dict(dict_obj, values)`**: Temporarily modify a dictionary

### spec and Autospec
- **`Mock(spec=ClassName)`**: Restrict attribute access to real interface
- **`create_autospec(ClassName)`**: Also validates argument signatures